# Gaia colour magnitude diagram (CMD) for HOYS Targets

Welcome to the CMD notebook for the HOYS LCO Project.\
Please see the video tutorial.

### Notebook tips:
* ***Shift + Enter on a code cell/block to run it and advance to the next cell.***
* Can re-run blocks out of order as long as the variables are already there
* If python kernal crashses or out of memory, use the reset button above to reset the kernal.

In [1]:
import time
import math
from matplotlib import *
from matplotlib.pyplot import *
import numpy as np
import pandas as pd
import astropy
from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
from astroquery.skyview import SkyView
import numpy.ma as ma
import os
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widget
from IPython.display import display,clear_output


In [ ]:
#for lab
%matplotlib widget
#for notebook/slides
#%matplotlib notebook #for notebook/slides
rcParams.update({'figure.max_open_warning': 50})

In [2]:
#load in isochrones
iso_table_1myr = pd.read_csv('iso/gaia_edr3_iso_1myr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)
iso_table_2myr = pd.read_csv('iso/gaia_edr3_iso_2myr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)
iso_table_3myr = pd.read_csv('iso/gaia_edr3_iso_3myr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)
iso_table_4myr = pd.read_csv('iso/gaia_edr3_iso_4myr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)
iso_table_5myr = pd.read_csv('iso/gaia_edr3_iso_5myr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)
iso_table_6myr = pd.read_csv('iso/gaia_edr3_iso_6myr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)
iso_table_7myr = pd.read_csv('iso/gaia_edr3_iso_7myr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)
iso_table_8myr = pd.read_csv('iso/gaia_edr3_iso_8myr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)
iso_table_9myr = pd.read_csv('iso/gaia_edr3_iso_9myr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)
iso_table_10myr = pd.read_csv('iso/gaia_edr3_iso_10myr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)
iso_table_20myr = pd.read_csv('iso/gaia_edr3_iso_20myr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)
iso_table_100myr = pd.read_csv('iso/gaia_edr3_iso_100myr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)
iso_table_1gyr = pd.read_csv('iso/gaia_edr3_iso_1gyr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)
iso_table_1gyr=iso_table_1gyr.query('Gmag >-2')
iso_list={'1gyr':iso_table_1gyr,'100myr':iso_table_100myr,'20myr':iso_table_20myr,'10myr':iso_table_10myr,'9myr':iso_table_9myr,'8myr':iso_table_8myr,
          '7myr':iso_table_7myr,'6myr':iso_table_6myr,'5myr':iso_table_5myr,'4myr':iso_table_4myr,'3myr':iso_table_3myr,'2myr':iso_table_2myr,'1myr':iso_table_1myr}

FileNotFoundError: [Errno 2] No such file or directory: 'iso/gaia_edr3_iso_1myr.txt'

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
#select HOYS target
target_sel=widget.Dropdown(
    options=[('NGC 1333', '03 29 02 +31 20 54'), 
             ('IC 348', '03 44 34 +32 09 48'), 
             ('Lambda – Ori Cluster','05 35 06 +09 56 00'),
             ('M42','05 35 17 -05 23 28'), 
             ('L 1641 N Cluster','05 36 19 -06 22 12'), 
             ('Sigma – Ori Cluster','05 38 45 -02 36 00'), 
             ('NGC 2068','05 46 46 +00 04 12'), 
             ('NGC 2244','06 31 55 +04 56 30'),
             ('NGC 2264','06 40 58 +09 53 42'),
             ('MWSC 3274','20 11 13 +37 27 00'),
             ('P Cyg','20 17 47 +38 01 59'),
             ('Berkeley 86','20 20 12 +38 41 24'),
             ('IC 5070','20 51 00 +44 22 00'),
             ('IC 1396 A','21 36 35 +57 30 36'),
             ('IC 1396 N','21 40 42 +58 16 06'),
             ('NGC 7129','21 42 56 +66 06 12'),
             ('IC 5146','21 53 29 +47 16 01'),
             ('Gaia19fct','07 09 21 -10 29 35')
            ],
    description='Target:',
)
display(target_sel)

Dropdown(description='Target:', options=(('NGC 1333', '03 29 02 +31 20 54'), ('IC 348', '03 44 34 +32 09 48'),…

In [5]:
c = SkyCoord(target_sel.value, unit=(u.hourangle, u.deg))
coords=[c.ra.value,c.dec.value]
radius=0.5
radius2=0.3

In [16]:
#query DSS2 IR image server
print('querying DSS2 image server for ',target_sel.label,' around coords:',c.to_string('hmsdms'))
img = SkyView.get_images(position=c,survey=['DSS2 IR'],coordinates='J2000',width=1*u.degree,height=1*u.degree)
b = img[0][0]

querying DSS2 image server for  NGC 1333  around coords: 03h29m02s +31d20m54s


In [19]:
#display image and make selections for position of Gaia query and cluster subset
def view_image(rad_sel=0.5,rad2_sel=0.25,ra_sel=c.ra.value,dec_sel=c.dec.value):
    global radius, radius2, coords
    radius=rad_sel
    radius2=rad2_sel
    fig, ax = subplots(figsize=(7,7))
    ax.imshow(b.data,cmap='gray',vmin=1e3, vmax=1.5e4,
              extent=[b.header['CRVAL1']-(b.header['NAXIS1']-b.header['CRPIX1'])*b.header['CDELT1'],
                                                               b.header['CRVAL1']+(b.header['NAXIS1']-b.header['CRPIX1'])*b.header['CDELT1'],
                                                               b.header['CRVAL2']+(b.header['NAXIS2']-b.header['CRPIX2'])*b.header['CDELT2'],
                                                               b.header['CRVAL2']-(b.header['NAXIS2']-b.header['CRPIX2'])*b.header['CDELT2']])
    fig.gca().invert_yaxis()
    circle = Circle((c.ra.deg,c.dec.deg),rad_sel,color='white',fill=False,linewidth=3)
    circle2 = Circle((ra_sel,dec_sel),rad2_sel,color='red',fill=False,linewidth=3)
    fig.gca().add_artist(circle)
    fig.gca().add_artist(circle2)
    ax.set_xlabel('RA [deg]')
    ax.set_ylabel('Dec [deg]')
    tight_layout()
    show()
    coords=[ra_sel,dec_sel]


interact(view_image,rad_sel=widget.FloatSlider(value=0.5,min=0.1,max=0.6,step=0.05,description='Query circle radius [pc]:',
                            layout={'width': '600px'},continuous_update=False,style = {'description_width': 'initial'}),
        rad2_sel=widget.FloatSlider(value=0.25,min=0.1,max=0.6,step=0.05,description='Cluster selection radius [pc]:',
                            layout={'width': '600px'},continuous_update=False,style = {'description_width': 'initial'}),
        ra_sel=widget.FloatSlider(value=c.ra.value,min=c.ra.value-0.5,max=c.ra.value+0.5,step=0.01,description='Cluster selection RA [deg]:',
                            layout={'width': '600px'},continuous_update=False,style = {'description_width': 'initial'}),
        dec_sel=widget.FloatSlider(value=c.dec.value,min=c.dec.value-0.5,max=c.dec.value+0.5,step=0.01,description='Cluster selection Dec [deg]:',
                            layout={'width': '600px'},continuous_update=False,style = {'description_width': 'initial'}))

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='Query circle radius [pc]:',…

<function __main__.view_image(rad_sel=0.5, rad2_sel=0.25, ra_sel=52.25833333333333, dec_sel=31.348333333333333)>

In [ ]:
#query Gaia catalogue
print('querying Gaia catalogue for ',target_sel.label,' around coords:',c.to_string('hmsdms'), 'with radius ',radius,' deg')

rad = u.Quantity(radius, u.deg)

MAIN_GAIA_TABLE = "gaiaedr3.gaia_source" # Select early Data Release 3
#MAIN_GAIA_TABLE = "gaiadr2.gaia_source"  # Reselect Data Release 2, default
Gaia.ROW_LIMIT=-1

j = Gaia.cone_search_async(c, rad,columns=[],table_name=MAIN_GAIA_TABLE)

r = j.get_results()
r

In [ ]:
#determine separation from cluster centre reference coords
cat=SkyCoord(r['ra'],r['dec'])
ref=SkyCoord(coords[0]*u.deg,coords[1]*u.deg)
sep = cat.separation(ref)
sep_ref=pd.Series((sep),name='sep_ref')
print('selecting from Gaia catalogue around ref coords:',np.round(coords,2), 'with radius ',radius2,' deg')
cluster_table_wide=r.to_pandas()
cluster_table_wide.parallax=cluster_table_wide.parallax-0.021 #Parallax zero point off-set for eDR3
cluster_table_wide=pd.concat([cluster_table_wide,sep_ref],axis=1)
#intial filtering of parallax with min G mag and colour selections
pzero=0.3 #ignore <0.3 parallax
perr=5 #set min parallax error
g_min=18
cluster_table_wide=cluster_table_wide.query(f'parallax > {pzero} and parallax_error < {perr} and phot_g_mean_mag < {g_min} and bp_g > -0.2 and g_rp < 2.0')
distance_eDR3=pd.Series((1/cluster_table_wide.parallax*10e2),name='distance_eDR3')
cluster_table_wide=pd.concat([cluster_table_wide,distance_eDR3],axis=1) #add distance column to each object
#calculate absolute G band magnitudes using distances
abs_phot_g_mean_mag_eDR3=pd.Series(cluster_table_wide['phot_g_mean_mag']-5*np.log10(cluster_table_wide['distance_eDR3']/10),name='abs_phot_g_mean_mag_eDR3')
cluster_table_wide=pd.concat([cluster_table_wide,abs_phot_g_mean_mag_eDR3],axis=1)

#select all stars within reference radius, 

cluster_table_all=cluster_table_wide.query(f'sep_ref < {radius2}')

parallax=np.median(cluster_table_all['parallax'])
print('median parallax of position selected region:',np.round(parallax,3),' mas')
print('median distane to position selected region:',np.round(1/parallax*1000,3),' pc')
dist=1/parallax*1000

#select main sequence stars from entire region
cluster_table_ms=cluster_table_wide.query(f'parallax > 2')

#### Plot CMD for entire region

In [ ]:
fig1, ax1 = subplots(1,1)
ax1.plot(cluster_table_wide['bp_rp'],cluster_table_wide['abs_phot_g_mean_mag_eDR3'],'k.',label='all query',markersize=0.5)
ax1.plot(cluster_table_all['bp_rp'],cluster_table_all['abs_phot_g_mean_mag_eDR3'],'r.',label='cluster position and magnitude query',markersize=5)
ax1.set_xlim(min(cluster_table_all['bp_rp'])-0.5,max(cluster_table_all['bp_rp'])+0.5)
ax1.set_ylim(max(cluster_table_all['abs_phot_g_mean_mag_eDR3'])+1,min(cluster_table_all['abs_phot_g_mean_mag_eDR3'])-1)
ax1.legend()
ax1.set_xlabel('BP-RP [mag]')
ax1.set_ylabel('G [mag]')
show()

#### Plot histogram of parallaxes and select window for cluster parallax

In [ ]:
def sel_par_range(drange=[0,2000],wbins=50,log=False):
    global cluster_table_par
    cluster_table_par=cluster_table_all.query(f'distance_eDR3 > {drange[0]} and distance_eDR3 < {drange[1]}')
    figure()
    data=cluster_table_all['distance_eDR3'][cluster_table_all['distance_eDR3']<2000]
    w = wbins
    n = math.ceil((max(data) - min(data))/w)
    hist(data,bins=n,label='position query',log=log,color='c')
    data2=cluster_table_par['distance_eDR3'][cluster_table_par['distance_eDR3']<2000]
    n2 = math.ceil((max(data2) - min(data2))/w)
    hist(data2,bins=n2,label='cluster parallax selection',log=log,color='r')
    xlabel('distance [pc]')
    ylabel('count')
    legend()
    show()
    
interact(sel_par_range, drange=widget.FloatRangeSlider(value=[0,2000],min=0,max=2000,step=1,description='Distance range [pc]:',
                                    readout_format='.0f',layout={'width': '600px'},style = {'description_width': 'initial'},
                                    continuous_update=False),
        wbins=widget.FloatSlider(value=50,min=10,max=100,step=1,description='bin width [pc]:',continuous_update=False,style = {'description_width': 'initial'}),
        log=widget.Checkbox(value=False,description='use log scale for count?'))


#### Plot CMD with parallax selection highlighted

In [ ]:
fig1, ax1 = subplots(1,1)
ax1.plot(cluster_table_all['bp_rp'],cluster_table_all['abs_phot_g_mean_mag_eDR3'],'c.',label='position selection')
ax1.plot(cluster_table_par['bp_rp'],cluster_table_par['abs_phot_g_mean_mag_eDR3'],'r.',label='parallax selection')
ax1.set_xlim(min(cluster_table_all['bp_rp'])-0.5,max(cluster_table_all['bp_rp'])+0.5)
ax1.set_ylim(max(cluster_table_all['abs_phot_g_mean_mag_eDR3'])+1,min(cluster_table_all['abs_phot_g_mean_mag_eDR3'])-1)
ax1.legend()
ax1.set_xlabel('BP-RP [mag]')
ax1.set_ylabel('G [mag]')
show()

#### Plot proper motions with parallax selection highlighted

In [ ]:
fig3, ax3 = subplots(1,1)
#ax3.plot(cluster_table_all['pmra'],cluster_table_all['pmdec'],'g.',label='all query')
#ax3.plot(cluster_table_par['pmra'],cluster_table_par['pmdec'],'r.',label='par  sel')
errorbar(cluster_table_all['pmra'],cluster_table_all['pmdec'], 
         xerr=cluster_table_all['pmra_error'], yerr=cluster_table_all['pmdec_error'], fmt=".", color="c",label='pos sel')
errorbar(cluster_table_par['pmra'],cluster_table_par['pmdec'], 
         xerr=cluster_table_par['pmra_error'], yerr=cluster_table_par['pmdec_error'], fmt=".", color="r",label='par sel')
xlim(-30,30)
ylim(-30,30)
ax3.legend()
ax3.set_xlabel('pmra [mas/yr]')
ax3.set_ylabel('pmdec [mas/yr]')
show()

##### zoom in on the above plot to set limits for next plot

In [ ]:
from scipy.stats import kde

pmdec_min, pmdec_max = ax3.get_ylim()
pmra_min, pmra_max = ax3.get_xlim()

cluster_table=cluster_table_par.query(f'pmra > {pmra_min} and pmra < {pmra_max} and pmdec > {pmdec_min} and pmdec < {pmdec_max}')

# select data
x = cluster_table['pmra']
y = cluster_table['pmdec']
 
# Evaluate a gaussian kde on a regular grid of nbins x nbins over data extents
nbins=100
k = kde.gaussian_kde([x,y])
xi, yi = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
zi = k(np.vstack([xi.flatten(), yi.flatten()]))
 
# Make the plot
fig5, ax5 = subplots(1,1)
ax5.plot(cluster_table['pmra'],cluster_table['pmdec'],'k.',label='all query',alpha=0.1)
ax5.pcolormesh(xi, yi, zi.reshape(xi.shape),norm=colors.PowerNorm(gamma=5),shading='auto',cmap='Spectral_r')

ax5.set_xlabel('pmra [mas/yr]')
ax5.set_ylabel('pmdec [mas/yr]')
show()

##### zoom in again on the above KDE plot to set limits for final selection

#### Plot CMD with parallax and proper motion selections

In [ ]:
pmdec_min, pmdec_max = ax5.get_ylim()
pmra_min, pmra_max = ax5.get_xlim()
cluster_table=cluster_table_par.query(f'pmra > {pmra_min} and pmra < {pmra_max} and pmdec > {pmdec_min} and pmdec < {pmdec_max}')


fig1, ax1 = subplots(1,1)
ax1.plot(cluster_table_all['bp_rp'],cluster_table_all['abs_phot_g_mean_mag_eDR3'],'c.',label='position selection')
ax1.plot(cluster_table['bp_rp'],cluster_table['abs_phot_g_mean_mag_eDR3'],'r.',label='cluster parallax and pm selection')
ax1.set_xlim(min(cluster_table_all['bp_rp'])-0.5,max(cluster_table_all['bp_rp'])+0.5)
ax1.set_ylim(max(cluster_table_all['abs_phot_g_mean_mag_eDR3'])+1,min(cluster_table_all['abs_phot_g_mean_mag_eDR3'])-1)
ax1.legend()
ax1.set_xlabel('BP-RP [mag]')
ax1.set_ylabel('G [mag]')
show()

In [ ]:
parallax=np.median(cluster_table['parallax'])
print('median parallax of selected cluster members:',np.round(parallax,3),' mas')
print('median distane to selected cluster members:',np.round(1/parallax*10e2,3),' pc')
dist=1/parallax*10e2

#### Overplot isochrones

In [ ]:
print('Ctrl/Cmd + Click to select isochrones to plot')
def plot_cmd(iso_sel=['100myr','10myr','1myr']):
    fig2, ax2 = subplots(1,1)
    ax2.plot(cluster_table['bp_rp'],cluster_table['abs_phot_g_mean_mag_eDR3'],'r.',label='Cluster',markersize=10)
    ax2.plot(cluster_table_ms['bp_rp'],cluster_table_ms['abs_phot_g_mean_mag_eDR3'],'k.',label='Main Sequence',markersize=1)

    for iso in iso_sel:
        ax2.plot(iso_list[iso]['G_BPmag']-iso_list[iso]['G_RPmag'],iso_list[iso]['Gmag'],'-',label=iso)

    ax2.set_xlim(min(cluster_table['bp_rp'])-0.5,max(cluster_table['bp_rp'])+0.5)
    ax2.set_ylim(max(cluster_table['abs_phot_g_mean_mag_eDR3'])+1,min(cluster_table['abs_phot_g_mean_mag_eDR3'])-1)
    ax2.legend()
    ax2.set_xlabel('BP-RP [mag]')
    ax2.set_ylabel('G [mag]')
    show()

interact(plot_cmd,iso_sel=widget.SelectMultiple(options=iso_list.keys(),value=['100myr','10myr','1myr'],
    description='Select Isochrones to plot:',style = {'description_width': 'initial'},rows=len(iso_list.keys())))

#### Position plot of identified cluster

In [ ]:
fig1, ax1 = subplots(1,1)
ax1.plot(cluster_table_wide['ra'],cluster_table_wide['dec'],'k.',label='all query',markersize=1)
ax1.plot(cluster_table['ra'],cluster_table['dec'],'r.',label='cluster pos, par and pm selection')
ax1.legend()
ax1.invert_xaxis()
ax1.set_ylabel('Dec [deg]')
ax1.set_xlabel('RA [deg]')
show()

#### Display Gaia catalogue data for selected cluster members

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('You selected ',len(cluster_table),' potential young stars.')
cluster_table[['ra','dec','parallax','pmra','pmdec','phot_g_mean_mag','bp_rp']]

In [ ]:
close('all')